# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import fairlearn.moments as moments
import fairlearn.classred as red
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define Regressor Class

In [16]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

class LogisticRegressionLearner():
    def __init__(self):
        self.model = None
    def fit(self, X, Y, W):
#         sqrtW = np.sqrt(W)
#         print('sqrtW shape: ', sqrtW.shape)
#         print('sqrtW: ', sqrtW)
#         matX = np.array(X) * sqrtW[:, np.newaxis]
#         vecY = Y * sqrtW
#         print('vecY shape: ', vecY.shape)
#         print('vecY: ', vecY)
        self.model = LogisticRegression(C=0.3, solver='liblinear', tol=1e-3, penalty='l1', verbose=1, max_iter=100000)
        self.model.fit(X, Y, sample_weight=W)

    def predict(self, X):
        return self.model.predict(X)

# Get Files and Inputs

In [3]:
import os
print(os.listdir('../input/fair-classification'))

['test_sample0.csv', 'test_no_income.csv', 'train.csv']


In [4]:
df = pd.read_csv('../input/fair-classification/train.csv')
submission_df = pd.read_csv('../input/fair-classification/test_no_income.csv')

In [5]:
df.shape

(35988, 15)

In [6]:
df.head()

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital gain,capital loss,hours per week,native-country,income
0,44,Local-gov,100479,Assoc-acdm,12,Married-civ-spouse,Prof-specialty,Husband,0,0,0,0,48,United-States,0
1,57,Private,477867,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,0,0,0,0,40,United-States,0
2,21,Private,410439,Some-college,10,Never-married,Sales,Own-child,0,0,0,0,24,United-States,0
3,21,Private,195532,HS-grad,9,Never-married,Adm-clerical,Unmarried,0,1,0,0,40,United-States,0
4,57,Self-emp-inc,125000,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,0,0,0,0,35,United-States,1


In [7]:
submission_df.shape

(8997, 14)

In [8]:
submission_df.head()

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital gain,capital loss,hours per week,native-country
0,56,Local-gov,264436,Some-college,10,Divorced,Exec-managerial,Not-in-family,0,1,0,0,40,United-States
1,37,Private,184556,Some-college,10,Divorced,Tech-support,Unmarried,0,1,0,0,40,United-States
2,64,Local-gov,96076,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,0,0,0,0,40,United-States
3,36,Federal-gov,192150,HS-grad,9,Divorced,Adm-clerical,Unmarried,0,1,0,0,40,United-States
4,30,?,147215,Some-college,10,Never-married,?,Own-child,0,1,0,0,30,United-States


In [9]:
# encode string data
df = pd.get_dummies(df)
submission_df = pd.get_dummies(submission_df)

# Organize data into training, testing, features(X), sensitive attributes(A), targets(y)

In [10]:
train_df, test_df = train_test_split(df, test_size=0.25)
X_train, A_train, y_train = train_df.drop(['income','gender'], axis=1), train_df['gender'], train_df['income']
X_test, A_test, y_test = test_df.drop(['income','gender'], axis=1), test_df['gender'], test_df['income']

# Using fairlearn to find best model

In [17]:
learner = LogisticRegressionLearner()
res_tuple = red.expgrad(X_train, A_train, y_train, learner,
                        cons=moments.DP(), eps=0.1)
res_tuple

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

ExgradResult(best_classifier=<function expgrad.<locals>.<lambda> at 0x1139ab400>, best_gap=1.3877787807814457e-16, classifiers=0     <__main__.LogisticRegressionLearner object at ...
1     <__main__.LogisticRegressionLearner object at ...
2     <__main__.LogisticRegressionLearner object at ...
3     <__main__.LogisticRegressionLearner object at ...
4     <__main__.LogisticRegressionLearner object at ...
5     <__main__.LogisticRegressionLearner object at ...
6     <__main__.LogisticRegressionLearner object at ...
7     <__main__.LogisticRegressionLearner object at ...
8     <__main__.LogisticRegressionLearner object at ...
9     <__main__.LogisticRegressionLearner object at ...
10    <__main__.LogisticRegressionLearner object at ...
11    <__main__.LogisticRegressionLearner object at ...
12    <__main__.LogisticRegressionLearner object at ...
13    <__main__.LogisticRegressionLearner object at ...
14    <__main__.LogisticRegressionLearner object at ...
dtype: object, weights=0     0.00

In [24]:
res['best_classifier']

<function fairlearn.classred.expgrad.<locals>.<lambda>>

In [26]:
res = res_tuple._asdict()
best_classifier = res['best_classifier']
best_classifier

<function fairlearn.classred.expgrad.<locals>.<lambda>>

# Evaluate Results

In [29]:
# Make Predictions
predictions = best_classifier(X_test)

In [40]:
predictions[predictions < 0.5] = 0
predictions[predictions >= 0.5] = 1

In [42]:
accuracy = accuracy_score(y_test, predictions) * 100
print('Accuracy: ', round(accuracy, 2), '%.')

Accuracy:  85.1 %.


In [49]:
# X_pred includes income columns that is predicted by the model
X_pred = test_df.drop('income', axis=1)
X_pred['income'] = predictions

In [50]:
A0 = len(X_pred.loc[X_pred['gender'] == 0])
A1 = len(X_pred.loc[X_pred['gender'] == 1])

Y1A0 = len(X_pred.loc[(X_pred['income'] == 1) & (X_pred['gender'] == 0)])
Y1A1 = len(X_pred.loc[(X_pred['income'] == 1) & (X_pred['gender'] == 1)])

In [51]:
Th = 0.1
DDP = np.abs((Y1A0 / A0) - (Y1A1 / A1))
bias = (7**(DDP - Th) - 1) if DDP > Th else 0
score = accuracy - bias
print(f'Accuracy: {accuracy}\nBias: {bias}\nScore: {score}')

Accuracy: 85.09503167722573
Bias: 0
Score: 85.09503167722573
